In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
import numpy as np

In [3]:
L_smooth = 1.
mu = 0.00001

Inductance = 1
R = 1

n_func = 2
# solver = "ca"
solver = "ca_canonical"
# solver = "ca_canonical_X"
# solver = "cvx_dccp"
# solver = "qcqp_sni"
# solver = "bisection_b"

# Ciropt problem

In [4]:
# problem = co.decomp_euler_consensus2(mu, L_smooth, R, Inductance)
# problem = co.admm_consensus(n_func, mu, L_smooth, R, Inductance)
# problem = co.admm_consensus_pep(mu, L_smooth, R, Inductance)
problem = co.admm_consensus2(mu, L_smooth, R, Inductance)
# problem = co.admm_consensus_proof(mu, L_smooth, R, Inductance)
problem.obj = problem.b + problem.d

# bounds = {  'b': {"ub": 0.1, "lb": 0.1},
#             'h': {"ub": 1, "lb": 0},
#             'alpha': {"ub": 1, "lb": 1},
#             'beta': {"ub": 1, "lb": 1},}

# bounds = {  'b': {"ub": 10, "lb": 0.},
#             'd': {"ub": 0, "lb": 0.},
#             'h': {"ub": 10, "lb": 0},
#             'alpha': {"ub": 1, "lb": -1},
#             'beta': {"ub": 1, "lb": -1},}

res, model = problem.solve(solver=solver, verbose=False)[:2]
ca_vars = problem.vars

# res = params = {  'b': 10,
#             'h': 5e-5,
#             'd': 0.,
#             'alpha': 0,
#             'beta': 1}
# b = problem.solve(solver=solver, freq=1, max_iters=30, params=params, debug=True)
# print(f"{b = }")
# res["b"] = b

dim_G=8, dim_F=6


In [5]:
res

{'b': 5.409503079179581e-08,
 'h': 1.01953592488584e-07,
 'd': 6.643726735184523e-08,
 'alpha': 0.0,
 'beta': 0.0}

In [6]:
# {'b': -1.8307775793323645e-06,
#  'h': 0.20720796869821878,
#  'd': 4.506477734433067e-07,
#  'alpha': 0.0,
#  'beta': 0.0}0

# PEP verification

In [7]:
# res = {'b': 1,
#         'h': 1,
#         'd': 0.,
#         'alpha': 0,
#         'beta': 1}

In [8]:
# problem = co.admm_consensus_pep(n_func, mu, L_smooth, R, Inductance, params=res)
problem = co.admm_consensus2(mu, L_smooth, R, Inductance, params=res)
# problem = co.decomp_euler_consensus2(mu, L_smooth, R, Inductance, params=res)
# problem = co.admm_consensus_proof(mu, L_smooth, R, Inductance, params=res)
diff = problem.solve(verbose=1, solver=cp.MOSEK)
print(f"{diff=}")

(PEPit) Setting up the problem: size of the main PSD matrix: 8x8
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (0 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
		 function 1 : Adding 6 scalar constraint(s) ...
		 function 1 : 6 scalar constraint(s) added
		 function 2 : Adding 6 scalar constraint(s) ...
		 function 2 : 6 scalar constraint(s) added
(PEPit) Setting up the problem: constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver


SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
import sympy as sp

In [ ]:
x1_1 = sp.symbols("x_1^(1)")
y1_1 = sp.symbols("y_1^(1)")
x2_1 = sp.symbols("x_2^(1)")
y2_1 = sp.symbols("y_2^(1)")
z_1 = sp.symbols("z^(1)")
i1_1 = sp.symbols("i_1^(1)")
i2_1 = sp.symbols("i_2^(1)")
i1_0 = sp.symbols("i_1^(0)")
i2_0 = sp.symbols("i_2^(0)")
L, R, h, M = sp.symbols("L R h M")
z_star, y1_star, y2_star = sp.symbols("z^\star y_1^\star y_2^\star")
f_star, f_1= sp.symbols("f^\star f(x^{(1)})")

In [ ]:
E_0 = (L/2) * (i1_0 - y1_star)**2 + (L/2) * (i2_0 - y2_star)**2
E_1 = (L/2) * (i1_1 - y1_star)**2 + (L/2) * (i2_1 - y2_star)**2

In [ ]:
diff = (E_1 - E_0).subs(i1_1, i1_0 + (h/L)*(z_1 - x1_1))
diff = diff.subs(i2_1, i2_0 + (h/L)*(z_1 - x2_1))

In [ ]:
diff

-L*(i_1^(0) - y_1^\star)**2/2 - L*(i_2^(0) - y_2^\star)**2/2 + L*(i_1^(0) - y_1^\star + h*(-x_1^(1) + z^(1))/L)**2/2 + L*(i_2^(0) - y_2^\star + h*(-x_2^(1) + z^(1))/L)**2/2

In [ ]:
sp.simplify(sp.expand(diff))

h*(2*L*(-i_1^(0)*x_1^(1) + i_1^(0)*z^(1) - i_2^(0)*x_2^(1) + i_2^(0)*z^(1) + x_1^(1)*y_1^\star + x_2^(1)*y_2^\star - y_1^\star*z^(1) - y_2^\star*z^(1)) + h*x_1^(1)**2 - 2*h*x_1^(1)*z^(1) + h*x_2^(1)**2 - 2*h*x_2^(1)*z^(1) + 2*h*z^(1)**2)/(2*L)

In [ ]:
sp.factor(sp.simplify(sp.cancel(sp.simplify(sp.expand(diff)))))

h*(-2*L*i_1^(0)*x_1^(1) + 2*L*i_1^(0)*z^(1) - 2*L*i_2^(0)*x_2^(1) + 2*L*i_2^(0)*z^(1) + 2*L*x_1^(1)*y_1^\star + 2*L*x_2^(1)*y_2^\star - 2*L*y_1^\star*z^(1) - 2*L*y_2^\star*z^(1) + h*x_1^(1)**2 - 2*h*x_1^(1)*z^(1) + h*x_2^(1)**2 - 2*h*x_2^(1)*z^(1) + 2*h*z^(1)**2)/(2*L)

In [ ]:
# Delta = R * (i1_0 - y1_1)**2 + R * (i2_0 - y2_1)**2 + (x1_1 - z_star) * (y1_1 - y1_star) + (x2_1 - z_star) * (y2_1 - y2_star)
Delta = R * (i1_0 - y1_1)**2 + R * (i2_0 - y2_1)**2 + f_1 - f_star - y1_star * (x1_1 - z_star) - y2_star *(x2_1 - z_star) 

In [ ]:
Delta

R*(i_1^(0) - y_1^(1))**2 + R*(i_2^(0) - y_2^(1))**2 + f(x^{(1)}) - f^\star - y_1^\star*(x_1^(1) - z^\star) - y_2^\star*(x_2^(1) - z^\star)

In [ ]:
F = diff/h + Delta - (h/(2*L))*(z_1 - x1_1)**2 - (h/(2*L))*(z_1 - x2_1)**2

In [ ]:
F2 = (z_1 - x1_1)*(i1_0 - y1_star) + (z_1 - x2_1)*(i2_0 - y2_star) \
    + R * (i1_0 - y1_1)**2 + R * (i2_0 - y2_1)**2 \
    + f_1 - f_star - y1_star * (x1_1 - z_star) - y2_star *(x2_1 - z_star) 

assert co.equal_expression(F, F2)
assert co.is_zero_expr(co.collapse_expression(F - F2))

In [ ]:
sp.collect(sp.simplify(sp.cancel(sp.expand(F))), h).coeff(h, 1)

0

In [ ]:
ineq1 = f_1 - f_star + y1_1 * (z_star - x1_1) + y2_1 * (z_star - x2_1) + \
    (1/(2*M)) * (y1_1 - y1_star)**2 + (1/(2*M)) * (y2_1 - y2_star)**2

ineq2 = f_star - f_1 - y1_star * (x1_1 - z_star) + y2_star * (x2_1 - z_star)

In [ ]:
ineq1

f(x^{(1)}) - f^\star + y_1^(1)*(-x_1^(1) + z^\star) + y_2^(1)*(-x_2^(1) + z^\star) + (y_1^(1) - y_1^\star)**2/(2*M) + (y_2^(1) - y_2^\star)**2/(2*M)

In [ ]:
ineq2

-f(x^{(1)}) + f^\star - y_1^\star*(x_1^(1) - z^\star) + y_2^\star*(x_2^(1) - z^\star)

In [ ]:
res = sp.simplify(sp.cancel(sp.expand(F - 2*ineq1 - 2 * ineq2))).subs(y2_star, -y1_star)
res = sp.simplify(res.subs(i1_0, -i2_0))
res

(M*(2*R*i_2^(0)**2 + 2*R*i_2^(0)*y_1^(1) - 2*R*i_2^(0)*y_2^(1) + R*y_1^(1)**2 + R*y_2^(1)**2 + f(x^{(1)}) - f^\star + i_2^(0)*x_1^(1) - i_2^(0)*x_2^(1) + 2*x_1^(1)*y_1^(1) + 2*x_1^(1)*y_1^\star + 2*x_2^(1)*y_1^\star + 2*x_2^(1)*y_2^(1) - 2*y_1^(1)*z^\star - 4*y_1^\star*z^\star - 2*y_2^(1)*z^\star) - y_1^(1)**2 + 2*y_1^(1)*y_1^\star - 2*y_1^\star**2 - 2*y_1^\star*y_2^(1) - y_2^(1)**2)/M

In [ ]:
sp.cancel(sp.collect(sp.collect(sp.collect(sp.collect(res.subs(R, 1).subs(L, 1), i2_0), y1_1), y2_1), y1_star))

(M*f(x^{(1)}) - M*f^\star + 2*M*i_2^(0)**2 + M*i_2^(0)*x_1^(1) - M*i_2^(0)*x_2^(1) + 2*M*i_2^(0)*y_1^(1) - 2*M*i_2^(0)*y_2^(1) + 2*M*x_1^(1)*y_1^(1) + 2*M*x_1^(1)*y_1^\star + 2*M*x_2^(1)*y_1^\star + 2*M*x_2^(1)*y_2^(1) + M*y_1^(1)**2 - 2*M*y_1^(1)*z^\star - 4*M*y_1^\star*z^\star + M*y_2^(1)**2 - 2*M*y_2^(1)*z^\star - y_1^(1)**2 + 2*y_1^(1)*y_1^\star - 2*y_1^\star**2 - 2*y_1^\star*y_2^(1) - y_2^(1)**2)/M

In [ ]:
res.subs(R, 1).subs(L, 1)

(M*(f(x^{(1)}) - f^\star + 2*i_2^(0)**2 + i_2^(0)*x_1^(1) - i_2^(0)*x_2^(1) + 2*i_2^(0)*y_1^(1) - 2*i_2^(0)*y_2^(1) + 2*x_1^(1)*y_1^(1) + 2*x_1^(1)*y_1^\star + 2*x_2^(1)*y_1^\star + 2*x_2^(1)*y_2^(1) + y_1^(1)**2 - 2*y_1^(1)*z^\star - 4*y_1^\star*z^\star + y_2^(1)**2 - 2*y_2^(1)*z^\star) - y_1^(1)**2 + 2*y_1^(1)*y_1^\star - 2*y_1^\star**2 - 2*y_1^\star*y_2^(1) - y_2^(1)**2)/M

In [ ]:
sp.collect(res.subs(R, 1).subs(L, 1), M)

(M*(f(x^{(1)}) - f^\star + 2*i_2^(0)**2 + i_2^(0)*x_1^(1) - i_2^(0)*x_2^(1) + 2*i_2^(0)*y_1^(1) - 2*i_2^(0)*y_2^(1) + 2*x_1^(1)*y_1^(1) + 2*x_1^(1)*y_1^\star + 2*x_2^(1)*y_1^\star + 2*x_2^(1)*y_2^(1) + y_1^(1)**2 - 2*y_1^(1)*z^\star - 4*y_1^\star*z^\star + y_2^(1)**2 - 2*y_2^(1)*z^\star) - y_1^(1)**2 + 2*y_1^(1)*y_1^\star - 2*y_1^\star**2 - 2*y_1^\star*y_2^(1) - y_2^(1)**2)/M